# Importing Libraries

In [1]:
import os
import random
import numpy as np
from dotenv import load_dotenv
from dataclasses import dataclass

# pytorch
import torch

# huggingface
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer

# Hyperparameters

In [ ]:
@dataclass
class CONFIG:
    # Debug
    debug: bool = False
    
    # Model
    model_id: str = "PathFinderKR/Waktaverse-Llama-3-KO-8B-Instruct"
    
    # Device
    device: str = None
    
    # Tokenizer parameters
    max_length: int = 8192
    padding: str = "do_not_pad"
    truncation: bool = True
    
    # Generation parameters
    num_return_sequences: int = 1
    max_new_tokens: int = 1024
    do_sample: bool = True
    temperature: float = 0.6
    top_p: float = 0.9
    repetition_penalty: float = 1.1
    
    # bitsandbytes parameters
    load_in_4bit: bool = True
    bnb_4bit_compute_dtype: torch.dtype = torch.bfloat16
    bnb_4bit_quant_type: str = "nf4"
    bnb_4bit_use_double_quant: bool = True
    
    # Seed
    seed: int = 101

# Reproducibility

In [ ]:
def set_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    print(f"Seed: {seed}")
    
set_seed(CONFIG.seed)

# Device

In [3]:
def configure_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        num_gpu = torch.cuda.device_count()
        print("> Running on GPU", end=' | ')
        print("Num of GPUs: ", num_gpu)
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
        print("> Running on MPS")
    else:
        device = torch.device("cpu")
        print("> Running on CPU")
    return device

CONFIG.device = configure_device()

Device = cuda:0


In [4]:
def configure_attn_implementation(device):
    if device == "cuda":
        if torch.cuda.get_device_capability()[0] >= 8: # Ampere, Ada, or Hopper GPUs
            attn_implementation = "flash_attention_2"
            torch_dtype = torch.bfloat16
        else:
            attn_implementation = "eager"
            torch_dtype = torch.float16
    else:
        attn_implementation = "eager"
        torch_dtype = torch.float32
    return attn_implementation, torch_dtype

CONFIG.attn_implementation, CONFIG.torch_dtype = configure_attn_implementation(CONFIG.device)

Attention Implementation = flash_attention_2


# Hugging Face

In [2]:
load_dotenv()
login(
    token=os.getenv("HUGGINGFACE_TOKEN"),
    add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/pathfinder/.cache/huggingface/token
Login successful


# Model

In [7]:
tokenizer = AutoTokenizer.from_pretrained(CONFIG.model_id)
streamer = TextStreamer(tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=CONFIG.load_in_4bit,
    bnb_4bit_compute_dtype=CONFIG.bnb_4bit_compute_dtype,
    bnb_4bit_quant_type=CONFIG.bnb_4bit_quant_type,
    bnb_4bit_use_double_quant=CONFIG.bnb_4bit_use_double_quant
)

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    CONFIG.model_id,
    device_map=CONFIG.device,
    attn_implementation=CONFIG.attn_implementation,
    torch_dtype=CONFIG.torch_dtype,
    quantization_config=quantization_config
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
print(model)
print(f"Number of parameters: {model.num_parameters() / 1e9:.2f}")

```LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=128256, bias=False)
)```

# Inference

In [12]:
# Llama-3 Template
def prompt_template(system, user):
    return (
        "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n"
        f"{system}<|eot_id|>"
        
        "<|start_header_id|>user<|end_header_id|>\n\n"
        f"{user}<|eot_id|>"
        
        "<|start_header_id|>assistant<|end_header_id|>\n\n"
    )

In [13]:
# Llama-2 Template
def prompt_template_2(system, user):
    return (
        "<s>[INST]<<SYS>>\n"
        f"{system}\n"
        "<</SYS>>\n\n"
        
        f"{user}[/INST]"
    )

In [14]:
def generate_response(system ,user):
    prompt = prompt_template(system, user)
    
    input_ids = tokenizer.encode(
        prompt,
        max_length=CONFIG.max_length,
        padding=CONFIG.padding,
        truncation=CONFIG.truncation,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(CONFIG.device)
    
    outputs = model.generate(
        input_ids=input_ids,
        pad_token_id=tokenizer.eos_token_id,
        num_return_sequences=CONFIG.num_return_sequences,
        max_new_tokens=CONFIG.max_new_tokens,
        do_sample=CONFIG.do_sample,
        temperature=CONFIG.temperature,
        top_p=CONFIG.top_p,
        repetition_penalty=CONFIG.repetition_penalty,
        streamer=streamer
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=False)

In [15]:
system_prompt = "다음 지시사항에 대한 응답을 작성해 주세요."
user_prompt = "피보나치 수열에 대해 설명해주세요."

In [16]:
response = generate_response(system_prompt, user_prompt)
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

다음 지시사항에 대한 응답을 작성해 주세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

피보나치 수열에 대해 설명해주세요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

피보나치 수열은 수학에서 자주 사용되는 수열 중 하나로, 0과 1로 시작하여 다음 항이 이전 두 항의 합으로 구성됩니다. 피보나치 수열은 유명한 수학자 레온 알렉산드로비치 피보나치가 제안했으며, 그의 이름을 따서 명명되었습니다. 이 수열은 자연수와 정수를 포함하며, 각 항은 이전 두 항의 합입니다. 예를 들어, 첫 번째 항은 0이고 두 번째 항은 1이며, 세 번째 항은 2이고 네 번째 항은 3입니다. 피보나치 수열은 순차적으로 증가하는 특징이 있지만, 숫자가 커질수록 점점 더 빠르게 증가합니다. 피보나치 수열은 다양한 분야에서 사용되며, 수학, 컴퓨터 과학, 생물학 등에서 중요한 역할을 합니다.<|eot_id|>
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

다음 지시사항에 대한 응답을 작성해 주세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

피보나치 수열에 대해 설명해주세요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

피보나치 수열은 수학에서 자주 사용되는 수열 중 하나로, 0과 1로 시작하여 다음 항이 이전 두 항의 합으로 구성됩니다. 피보나치 수열은 유명한 수학자 레온 알렉산드로비치 피보나치가 제안했으며, 그의 이름을 따서 명명되었습니다. 이 수열은 자연수와 정수를 포함하며, 각 항은 이전 두 항의 합입니다. 예를 들어, 첫 번째 항은 0이고 두 번째 항은 1이며, 세 번째 항은 2이고 네 번째 항은 3입니다. 피